In [1]:
### load packages
import itertools
from itertools import compress
import pdb

import time
from datetime import timedelta, datetime
from os.path import join
import os
from os import listdir
from os.path import isfile, join
import csv
from timeit import default_timer as timer

import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import figure
from matplotlib.ticker import PercentFormatter
%matplotlib inline

from mne.filter import filter_data

import scipy
from scipy.signal import welch, decimate, periodogram, find_peaks
from scipy import signal
from scipy import stats
from scipy.stats import pearsonr, mannwhitneyu, spearmanr, ranksums, ttest_ind
from statsmodels.stats.multitest import multipletests
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats.anova import AnovaRM
from statistics import mode
import math  
import random 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import LeaveOneGroupOut, cross_val_predict, cross_val_score, KFold, train_test_split, StratifiedKFold, GridSearchCV 
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, auc, f1_score, precision_recall_curve

import sys

In [2]:
print('Pandas:',pd.__version__)
print('Numpy:',np.__version__)
print('Scipy:',scipy.__version__)
# Pandas: 1.1.3
# Numpy: 1.19.1
# Scipy: 1.5.2

Pandas: 1.1.3
Numpy: 1.19.1
Scipy: 1.5.2


# Notebook to Perform Supervised Classification Analysis

<p> <strong>Part 1:</strong> Loads in extracted features in .csv files per patient. Uses clinical scores from updrs .csv table to select patients to include in analysis based on clinical fluctuations pre vs post medication. </p>
<p> <strong>Part 2:</strong> Executes classification analysis.  </p>
<p> <strong>Part 3:</strong> Executes classification analysis. Different functions for individual vs group models, optional to include activity filter to remove inactive feature-windows. Seperate function for true predictions and permutation creation. Significancy testing in seperate Results notebook.  </p>
<p> <strong>Part 4:</strong> Sub analysis analyzing the influence of the number of patients used as training data in the group model.  </p>

In [3]:
# define path, working directory which contains filtered accelerometer files
notebook_path = os.getcwd()
path = os.path.dirname(notebook_path)
images_path = "/Users/jeroenhabets/Starr Lab Dropbox/jeroen habets/PHD werkmap/UCSF time/Sensor-project/analysis images/"
os.chdir(path)
print('Path and working directory defined as %s' %path)


Path and working directory defined as /Users/jeroenhabets/Research/pd_sensors/brady_reallife


## Part 1: Feature Loading
Previsouly filtered data is loaded in to dataframes per patient-side.

In [4]:
def loadUPDRSscores():
    '''
    Input:
    Load in existing updrsScores table
    
    Returns:
    - updrsScores: dataframe with clinical scores of all patients
    
    '''
    # Read in updrs file
    updrsScores = pd.read_csv(os.path.join(path,'data','updrsScores.csv'))
    # convert int to strings for PtId's
    updrsScores['PtId'] = updrsScores['PtId'].astype(str)
    # add missing zeros in front of PtId
    for row in np.arange(updrsScores.shape[0]):
        updrsScores.at[row,'PtId'] = updrsScores.loc[row]['Record Id'][3:]
    del(updrsScores['Record Id'])
    updrsScores = updrsScores.set_index('PtId')
    
    # create list for ON and OFF for subscore-lists
    OFF_sublists = {
        'leftHandBradyOFF': ['OFF_UPDRS_3_3c','OFF_UPDRS_3_4b','OFF_UPDRS_3_5b','OFF_UPDRS_3_6b',],
        'rightHandBradyOFF': ['OFF_UPDRS_3_3b','OFF_UPDRS_3_4a','OFF_UPDRS_3_5a','OFF_UPDRS_3_6a',],
        'bradyBodyOFF': ['OFF_UPDRS_3_14'],
        'leftHandTremorOFF': ['OFF_UPDRS_3_15b','OFF_UPDRS_3_16b','OFF_UPDRS_3_17b',],
        'rightHandTremorOFF': ['OFF_UPDRS_3_15a','OFF_UPDRS_3_16a','OFF_UPDRS_3_17a',],
        'gaitLegsOFF': ['OFF_UPDRS_3_7a','OFF_UPDRS_3_7b','OFF_UPDRS_3_8a','OFF_UPDRS_3_8b','OFF_UPDRS_3_9','OFF_UPDRS_3_10','OFF_UPDRS_3_11'],
        'postureOFF': ['OFF_UPDRS_3_12','OFF_UPDRS_3_13'],
        'facialOFF': ['OFF_UPDRS_3_1','OFF_UPDRS_3_2',]}
    
    ON_sublists = {
        'leftHandBradyON': ['ON_UPDRS_3_3c', 'ON_UPDRS_3_4b','ON_UPDRS_3_5b','ON_UPDRS_3_6b',],
        'rightHandBradyON': ['ON_UPDRS_3_3b','ON_UPDRS_3_4a','ON_UPDRS_3_5a','ON_UPDRS_3_6a',],
        'bradyBodyON': ['ON_UPDRS_3_14'],
        'leftHandTremorON': ['ON_UPDRS_3_15b','ON_UPDRS_3_16b','ON_UPDRS_3_17b',],
        'rightHandTremorON': ['ON_UPDRS_3_15a','ON_UPDRS_3_16a','ON_UPDRS_3_17a',],
        'gaitLegsON': ['ON_UPDRS_3_7a','ON_UPDRS_3_7b','ON_UPDRS_3_8a','ON_UPDRS_3_8b','ON_UPDRS_3_9','ON_UPDRS_3_10','ON_UPDRS_3_11'],
        'postureON': ['ON_UPDRS_3_12','ON_UPDRS_3_13'],
        'facialON': ['ON_UPDRS_3_1','ON_UPDRS_3_2',]}

    for l,(off, on) in enumerate(zip(OFF_sublists.keys(),ON_sublists.keys())):
        diffList = []
        relative_diffList = []
        for pt in updrsScores.index:
            diffScore = np.sum(updrsScores.loc[pt][OFF_sublists[off]]) - np.sum(updrsScores.loc[pt][ON_sublists[on]]) # calculate difference between off and on scores
            diffList.append(diffScore)
            relative_diffList.append(diffScore / (len(OFF_sublists[off])*4)) # take sum-difference, normalize to list-length by dividing by potential total score
        # create name for column
        rel_colName = off[:-3] + '_%diff'
        colName = off[:-3] + '_diff'
        updrsScores.insert(loc=l*2, value=diffList, column=colName)
        updrsScores.insert(loc=l*2 +1, value=relative_diffList, column=rel_colName)
    
    # create list with patients who have more bradykinesia fluctuation on left side
    affectedSides = []
    for pt in updrsScores.index:
        if updrsScores.loc[pt]['leftHandBrady_diff'] > updrsScores.loc[pt]['rightHandBrady_diff']:
            affectedSides.append('Left')
        else:
            affectedSides.append('Right')
    updrsScores.insert(loc=0, value=affectedSides, column='Side')
    
    return updrsScores

In [5]:
def loadFeatures(minBradyDiff, updrsScores, windowLen=60):
    '''
    Input:
    - minBradyDiff = minimal difference in brady-updrs-subscores to include in analysis
    - updrsScores: dataframe with clinical scores of all patients
    
    Returns:
    - accData: one dictionary with accData, each patient has a dictionary per side, 
    containing pre and post session.
    
    '''
    
    # select patients to involve in analysis
    selectedIDs = []
    for pt in updrsScores.index:
        if updrsScores.loc[pt]['leftHandBrady_diff'] > minBradyDiff:
            selectedIDs.append(pt)
        elif updrsScores.loc[pt]['rightHandBrady_diff'] > minBradyDiff:
            selectedIDs.append(pt)
        else:
            continue
    # removing 022 and 080 if included based on clinical difference, because data is not large enough for holdout
    for pt in ['022','080']:
        if pt in selectedIDs:
            selectedIDs.remove(pt)
    # dictionary for all acc data
    features = {}
    for pt in selectedIDs:
        features[pt] = pd.read_csv(os.path.join(path,'data','features_oct20/%s_%isec_features.csv' % (pt,windowLen)))
    
    return features
        

In [6]:
updrsScores = loadUPDRSscores()
features = loadFeatures(minBradyDiff=0.5, updrsScores=updrsScores, windowLen=60)

## Part 2: Individual Model Classification (incl. activity-filter sub analysis)

Splits data in to x and y dataframes per patient.
Uses always same hyperparameters, no hyperparameter optimization.

Includes:
- Balanced data regarding pre and post medication in training and test data
- Stratified, continuous selection of hold-out data: Takes timeblocks of 10% in pre-medication timespan, and 10% in post-medication timespan as holdout-validation data.
- Alternative data splitting method: <em>'Distributed data splitting'</em>, uses 2% of every 10-% data block as hold-out, the other 8% as trainin data. (0-8% is training, 8-10% is test, 10-18% is training, 18-20 is test, etc.)

In [43]:
### Function to perform true predictions for individual model (02DEC20)

def indiv_classification_true(clsf, features, feat_select, act_filter, save):
    '''
    Function calculates classification-outcomes for individual model approach with SVM classifier.
    Input: 
    - clsf: type of classification function: aupport vector: 'SV' or random forest: 'RF'
    - features-dataframe containing all patients, all 60-sec features, and medication-states.
    - feat_select: 'all' to include all features as predictors, use 'svm' to only include signal
    vector magnitude features, '4' only includes the selected 4 main features.
    - act_filter: defines activity filter or not
    - save: has to be binary, determining whether outcomes are written to results-folder.
    
    Output: table containing all 41 outcomes from different splits, per patient.
    
    Data is balanced 50/50 between pre- and post-medication moments.
    At least 40 minutes of pre and post are present in features (selected in feature extraction).
    Number of folds for cross-validation is individually adjusted to create folds of +/- 10 minutes.
    
    Data-splitting is adjusted: 2 minutes before and after test-data are not included in training data.
    To increase statistical power, 41 different datasplits are performed per patient. Data-split #0 takes % 0-10,
    and % 50-60 of features as test-data, #1 takes % 1-11, and % 51-61, ..., #40 takes % 40-50, and % 90-100 as
    test-data. The remaining data is always trainings-data (except for the neighbouring 2 minutes).
    
    SVM classifier using hyperparameters: C 10, gamma 0.001, kernel rbf.

    Fits once the model on the real X_train[pt] and y_train[pt]. This is fitting on individualized data.
    Calculates for this one real trial different performance metrics.

    '''
    starttime = timer()

## DATA PREP for individual-holdout validation
    pts = list(features.keys())
    keys = list(features[pts[0]].keys())
    if feat_select == 'all':
        sel = [k != 'medState' for k in keys]
        preds = list(compress(keys,sel))  
    elif feat_select == 'SVM':
        sel = [k[:3]=='SVM' for k in keys]
        preds = list(compress(keys,sel))
    elif feat_select == '4':
        sel = [k in ['SVM_maxAcc','SVM_coefVar','SVM_RMS','SVM_specPow_totalu4Hz'] for k in keys]
        preds = list(compress(keys,sel))
    
    metricNames = ['Accuracy', 'AUROC','F1_score', 'Precision', 'Recall', 'FPR'] # define metrics to check significancy for
    allpreds = pd.DataFrame(index=np.arange(41), columns=[pt+'_'+m for pt in pts for m in metricNames]) # 2d-df to save every pred per pt, per split
    ptDataSizes = pd.DataFrame(index=pts, columns=['n_ft_rows'])
    for pt in pts:
        trueLists = {} # list to store values during calculation
        for metric in metricNames:
            trueLists[metric]=[]
        
        if act_filter:
            presize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==0)
            postsize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==1)
            minSize=np.min([presize,postsize]) # minimal number of rows per med-state, after actvity filter
            # selecting only rows with SVM_variance above threshold
            actData = features[pt][features[pt]['SVM_variance'] > 0.3].reset_index(drop=True)
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predData = pd.concat([actData[actData['medState']==0][:minSize],actData[actData['medState']==1][:minSize]]).reset_index(drop=True)
        
        else:
            # minSize is shortest length of samples(rows) per pt-side of dataframe, on or off samples
            minSize = min(sum(features[pt]['medState']==0),sum(features[pt]['medState']==1))
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predData = pd.concat([features[pt][features[pt]['medState']==0][:minSize],features[pt][features[pt]['medState']==1][:minSize]]).reset_index(drop=True)
        ptDataSizes.loc[pt]['n_ft_rows'] = predData.shape[0]    
        # standarize data 
        scaler = StandardScaler()
        scaler.fit(predData[predData['medState']==0][preds]) # standardized only on pre-medication data

        # create x and y dataset for this patient
        x = pd.DataFrame(data = scaler.transform(predData[preds]), columns = preds)
        y = predData['medState']
        
        ### real prediction with accurate medication-labels (y) ###
        '''
        leading to 41 different outcome values for true-labels
        data splitting; creating 41 separate distributions of training/test splits with more temporal independence 
        first create individual dictionaries with row-numbers corresponding with percentage of data to split data
        
        there will be 41 different data-splits using the row-numbers corresponding to the percentile of data
        the data-splits will start at row 0, 1, 2, ..., until 40
        for every data-split the real-prediction will be performed, and a n-permutation-test will be performed to determine the significancy
        ''' 
        # dictionary of rownumbers corresponding with % of data
        rows = {}
        for r in np.arange(101):
            rows[r] = int(r/100*x.shape[0])
        
        ## First, Every iteration starts with data splitting, thereafter prediction
        for n_split in np.arange(41): # test with 20-fold cv, every second percentage new data split
            n_split = n_split # *2for test with 20-fold cv
            X_test = pd.concat([x[rows[n_split]:rows[n_split+10]],x[rows[n_split+50]:rows[n_split+60]]], ignore_index=True)
            y_test = pd.concat([y[rows[n_split]:rows[n_split+10]],y[rows[n_split+50]:rows[n_split+60]]], ignore_index=True)

            if n_split < 3: # for splits 0-1-2, no rows before the test-split are selected for training
                X_train = pd.concat([x[rows[n_split+12]:rows[50]],x[rows[n_split+62]:rows[100]]], ignore_index=True)
                y_train = pd.concat([y[rows[n_split+12]:rows[50]],y[rows[n_split+62]:rows[100]]], ignore_index=True)
                
            elif n_split > 37: # for splits 38-39-40, no minutes after the test-split are selected for training
                X_train = pd.concat([x[rows[0]:rows[n_split-2]],x[rows[50]:rows[n_split+48]]], ignore_index=True)
                y_train = pd.concat([y[rows[0]:rows[n_split-2]],y[rows[50]:rows[n_split+48]]], ignore_index=True)
                
            else: # minutes before and after the test-split are included in training-split
                X_train_a = pd.concat([x[rows[n_split+12]:rows[50]],x[rows[n_split+62]:rows[100]]], ignore_index=True)
                X_train_b = pd.concat([x[rows[0]:rows[n_split-2]],x[rows[50]:rows[n_split+48]]], ignore_index=True)
                X_train = pd.concat([X_train_a,X_train_b], ignore_index=True)
                y_train_a = pd.concat([y[rows[n_split+12]:rows[50]],y[rows[n_split+62]:rows[100]]], ignore_index=True)
                y_train_b = pd.concat([y[rows[0]:rows[n_split-2]],y[rows[50]:rows[n_split+48]]], ignore_index=True)
                y_train = pd.concat([y_train_a,y_train_b], ignore_index=True)

            ## Second, prediction part, define classifier
            if clsf == 'SV': # check which part can be used for randomforest
                sv = SVC(C=10, kernel='rbf', gamma=0.001, probability=True) 
                sv.fit(X_train, y_train) # fitting on 80% individual data, with true labels
                y_preds = sv.predict(X_test) # predicting
                y_probas = pd.DataFrame(data = sv.predict_proba(X_test), columns = sv.classes_) 
            elif clsf == 'RF':
                rf = RandomForestClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2,n_estimators=100)
                rf.fit(X_train, y_train) # fitting on 80% individual data, with true labels
                y_preds = rf.predict(X_test) # predicting
                y_probas = pd.DataFrame(data = rf.predict_proba(X_test), columns = rf.classes_) 
            y_true = y_test
            tn, fp, fn, tp = confusion_matrix(y_true,y_preds).ravel()
            # add outcomes to dedicated lists
            trueLists['Accuracy'].append(accuracy_score(y_true,y_preds))
            trueLists['AUROC'].append(roc_auc_score(y_true, y_probas[1]))
            trueLists['F1_score'].append(f1_score(y_true,y_preds))
            trueLists['Precision'].append(precision_score(y_true, y_preds)) # precision/PPV
            trueLists['Recall'].append(recall_score(y_true, y_preds)) # sensitivity/recall/TPR
            trueLists['FPR'].append(fp / (fp+tn)) # false-positive, false-alarm rate

        for metric in metricNames: # all data-splits for real data performed, save values in dataframe
            allpreds[pt+'_'+metric] = trueLists[metric]
            
        print(pt,'all predictions performed')
    
    endtime = timer()
    timePassed = (endtime-starttime)/60 # gives time in minutes
    
    if save == True:
        if act_filter == False:
            allpreds.to_csv(os.path.join(path,'results','indiv_preds_41splits_%s_%sfts_c10g001.csv' % (clsf,feat_select)), header=True, index=True)
#             ptDataSizes.to_csv(os.path.join(path,'results','indiv_preds_dataSizes_all.csv'), header=True, index=True)
        elif act_filter == True:
            allpreds.to_csv(os.path.join(path,'results','indiv_preds_41splits_%s_%sfts_c10g001_ACTFILTER.csv' % (clsf,feat_select)), header=True, index=True)
#             ptDataSizes.to_csv(os.path.join(path,'results','indiv_preds_dataSizes_ACTFILTER.csv'), header=True, index=True)    
    print('Minutes passed: %.1f' % timePassed)
    
    
    return allpreds



In [44]:
### EXECUTE REAL PREDICTIONS FOR INDIVIDUAL MODEL
# repeat for all 4 combinations (features and activity filter)
for f_sel in ['4','all']:
    for act in [False, True]:
        indiv_classification_true(clsf='RF', features=features, feat_select=f_sel, 
                                  act_filter=act, save=True)


002 all predictions performed
012 all predictions performed
013 all predictions performed
014 all predictions performed
015 all predictions performed


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


017 all predictions performed
018 all predictions performed
023 all predictions performed
024 all predictions performed
038 all predictions performed
039 all predictions performed
043 all predictions performed
047 all predictions performed
051 all predictions performed
054 all predictions performed
058 all predictions performed
063 all predictions performed
065 all predictions performed
079 all predictions performed
090 all predictions performed
Minutes passed: 2.1
002 all predictions performed
012 all predictions performed
013 all predictions performed
014 all predictions performed
015 all predictions performed
017 all predictions performed
018 all predictions performed
023 all predictions performed
024 all predictions performed
038 all predictions performed
039 all predictions performed
043 all predictions performed
047 all predictions performed
051 all predictions performed
054 all predictions performed


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


058 all predictions performed
063 all predictions performed
065 all predictions performed
079 all predictions performed
090 all predictions performed
Minutes passed: 2.0
002 all predictions performed
012 all predictions performed
013 all predictions performed


...

014 all predictions performed
015 all predictions performed


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


017 all predictions performed
018 all predictions performed
023 all predictions performed
024 all predictions performed
038 all predictions performed
039 all predictions performed
043 all predictions performed
047 all predictions performed
051 all predictions performed
054 all predictions performed
058 all predictions performed
063 all predictions performed
065 all predictions performed
079 all predictions performed
090 all predictions performed
Minutes passed: 2.0
002 all predictions performed
012 all predictions performed
013 all predictions performed
014 all predictions performed
015 all predictions performed
017 all predictions performed
018 all predictions performed
023 all predictions performed
024 all predictions performed
038 all predictions performed
039 all predictions performed
043 all predictions performed
047 all predictions performed
051 all predictions performed
054 all predictions performed


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


058 all predictions performed
063 all predictions performed
065 all predictions performed
079 all predictions performed
090 all predictions performed
Minutes passed: 1.9


In [47]:
### Function to perform permutations for individual model

def indiv_classification_perm(clsf, features, act_filter, n_perm, save):
    '''
    Function calculates classification-outcomes for individual model approach with SVM classifier.
    Input: 
    Input: 
    - clsf: type of classification function: aupport vector: 'SV' or random forest: 'RF'
    - features: dataframe containing all patients, all 60-sec features, and medication-states.
    - feat_select: 'all' to include all features as predictors, use 'svm' to only include signal
    vector magnitude features, '4' only includes the selected 4 main features.
    - act_filter: defines activity filter or not
    - n_perm defines number of permutations to perform.
    - save: has to be binary, determining whether outcomes are written to results-folder.
    
    Output: table with mean outcome of individual model per patient (mean is over the 41 possible data-splits),
    including p-values of permutation-test. 
    
    Data is balanced 50/50 between pre- and post-medication moments.
    At least 40 minutes of pre and post are present in features (selected in feature extraction).
    Number of folds for cross-validation is individually adjusted to create folds of +/- 10 minutes.
    
    Data-splitting is adjusted: 2 minutes before and after test-data are not included in training data.
    To increase statistical power, 41 different datasplits are performed per patient. Data-split #0 takes % 0-10,
    and % 50-60 of features as test-data, #1 takes % 1-11, and % 51-61, ..., #40 takes % 40-50, and % 90-100 as
    test-data. The remaining data is always trainings-data (except for the neighbouring 2 minutes).
    
    SVM classifier using hyperparameters: C 10, gamma 0.001, kernel rbf.

    Fits once the model on the real X_train[pt] and y_train[pt]. This is fitting on individualized data.
    Calculates this n_permutations times, with randomized y_train labels. This is model
     training with random input. For every random trial the outcome scores are calculated 
     and stored in the 'bins'. These bins represent all random performance values.
     P-value can be calculated by: 
     (# of permutations performing better then actual score + 1) / (# of permutations + 1)
     Based on sci-kit permutation_test_score, source: http://www.jmlr.org/papers/volume11/ojala10a/ojala10a.pdf

     Overview of predictive metrics: https://en.wikipedia.org/wiki/F1_score

    '''
    starttime = timer()

    pts = list(features.keys())
    keys = list(features[pts[0]].keys())
    ft_sel = [k != 'medState' for k in keys]
    preds = list(compress(keys,ft_sel))      
    metricNames = ['Accuracy', 'AUROC','F1_score', 'Precision', 'Recall', 'FPR'] # define metrics to check significancy for
    allperms = pd.DataFrame(index=np.arange(n_perm), columns=[pt+'_'+m for pt in pts for m in metricNames]) # 2d-df to save every pred per pt, per split
    for pt in pts:        
        if act_filter:
            presize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==0)
            postsize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==1)
            minSize=np.min([presize,postsize]) # minimal number of rows per med-state, after actvity filter
            # selecting only rows with SVM_variance above threshold
            actData = features[pt][features[pt]['SVM_variance'] > 0.3].reset_index(drop=True)
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predData = pd.concat([actData[actData['medState']==0][:minSize],actData[actData['medState']==1][:minSize]]).reset_index(drop=True) 
        else:
            # minSize is shortest length of samples(rows) per pt-side of dataframe, on or off samples
            minSize = min(sum(features[pt]['medState']==0),sum(features[pt]['medState']==1))
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predData = pd.concat([features[pt][features[pt]['medState']==0][:minSize],features[pt][features[pt]['medState']==1][:minSize]]).reset_index(drop=True)

        # standarize data 
        scaler = StandardScaler()
        scaler.fit(predData[predData['medState']==0][preds]) # standardized only on pre-medication data
        # create x and y dataset for this patient
        x = pd.DataFrame(data = scaler.transform(predData[preds]), columns = preds)
        y = predData['medState']
        '''
        For every round of permutation, 41 different data-splits will be predicted,
        leading to 41 different outcome values per permutation round, in total: n_datasplits*n_perm.
        Original x and y, and rows-dictionary are unchanged and can be used.
        ''' 
        rows = {} # dictionary of rownumbers corresponding with % of data
        for r in np.arange(101):
            rows[r] = int(r/100*x.shape[0])
        for p in np.arange(n_perm): ## Looping over number of permutation to perform, first creating permuted data-set
            perm_scores = {}
            for metric in metricNames:
                perm_scores[metric]=[]
            random.seed(p) # set structured different random seed for every shuffle loop
            y_random = list(y.values) # set true medication-labels to shuffle
            random.shuffle(y_random) # shuffles labels, x and y_random are now data to use
            y_rand = pd.DataFrame(data=y_random, columns=['med'])
            y_random = y_rand['med']
            ## Now, first create data-split, then perform prediction and add values to lists
            for n_split in np.arange(41): # test with 20-fold cv, every second percentage new data split
                n_split = n_split # *2 for test with 20-fold cv
                X_test = pd.concat([x[rows[n_split]:rows[n_split+10]],x[rows[n_split+50]:rows[n_split+60]]], ignore_index=True)
                y_test = pd.concat([y_random[rows[n_split]:rows[n_split+10]],y_random[rows[n_split+50]:rows[n_split+60]]], ignore_index=True)
                if n_split < 3: # for splits 0-1-2, no rows before the test-split are selected for training
                    X_train = pd.concat([x[rows[n_split+12]:rows[50]],x[rows[n_split+62]:rows[100]]], ignore_index=True)
                    y_train = pd.concat([y_random[rows[n_split+12]:rows[50]],y_random[rows[n_split+62]:rows[100]]], ignore_index=True)
                elif n_split > 37: # for splits 38-39-40, no minutes after the test-split are selected for training
                    X_train = pd.concat([x[rows[0]:rows[n_split-2]],x[rows[50]:rows[n_split+48]]], ignore_index=True)
                    y_train = pd.concat([y_random[rows[0]:rows[n_split-2]],y_random[rows[50]:rows[n_split+48]]], ignore_index=True)
                else: # minutes before and after the test-split are included in training-split
                    X_train_a = pd.concat([x[rows[n_split+12]:rows[50]],x[rows[n_split+62]:rows[100]]], ignore_index=True)
                    X_train_b = pd.concat([x[rows[0]:rows[n_split-2]],x[rows[50]:rows[n_split+48]]], ignore_index=True)
                    X_train = pd.concat([X_train_a,X_train_b], ignore_index=True)
                    y_train_a = pd.concat([y_random[rows[n_split+12]:rows[50]],y_random[rows[n_split+62]:rows[100]]], ignore_index=True)
                    y_train_b = pd.concat([y_random[rows[0]:rows[n_split-2]],y_random[rows[50]:rows[n_split+48]]], ignore_index=True)
                    y_train = pd.concat([y_train_a,y_train_b], ignore_index=True)
                ## Second, prediction part
                if clsf == 'SV':
                    sv = SVC(C=10, kernel='rbf', gamma=0.001, probability=True) # define classifier as indiv-optimized class
                    sv.fit(X_train, y_train) # fitting on 80% individual data, with true labels
                    y_preds = sv.predict(X_test) # predicting
                    y_probas = pd.DataFrame(data = sv.predict_proba(X_test), columns = sv.classes_) 
                elif clsf == 'RF':
                    rf = RandomForestClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2,n_estimators=100)
                    rf.fit(X_train, y_train) # fitting on 80% individual data, with true labels
                    y_preds = rf.predict(X_test) # predicting
                    y_probas = pd.DataFrame(data = rf.predict_proba(X_test), columns = rf.classes_) 
                y_true = y_test
                tn, fp, fn, tp = confusion_matrix(y_true,y_preds).ravel()
                # Third, add outcomes to dedicated lists
                perm_scores['Accuracy'].append(accuracy_score(y_true,y_preds))
                perm_scores['AUROC'].append(roc_auc_score(y_true, y_probas[1]))
                perm_scores['F1_score'].append(f1_score(y_true,y_preds))
                perm_scores['Precision'].append(precision_score(y_true, y_preds)) # precision/PPV
                perm_scores['Recall'].append(recall_score(y_true, y_preds)) # sensitivity/recall/TPR
                perm_scores['FPR'].append(fp / (fp+tn)) # false-positive, false-alarm rate
                # end of one data-split, in one permuted data-split
            for metric in metricNames: # at end of 41-splits in one permutations, save mean scores in outcome df
                allperms.loc[p][pt+'_'+metric] = np.mean(perm_scores[metric])
            
        print(pt,'all predictions ready')
        ptcols = list(compress(allperms.keys(),[pt in colname for colname in list(allperms.keys())]))
        ptperms = allperms[ptcols]
        if save == True:
            if act_filter == False:
                ptperms.to_csv(os.path.join(path,'results','perms','temp',
                                             '%s_indiv_%s_allfts_%iperms.csv' % (pt,clsf,n_perm)), header=True, index=True)
            elif act_filter == True:
                ptperms.to_csv(os.path.join(path,'results','perms','temp',
                                             '%s_indiv_%s_allfts_actfilter_%iperms.csv' % (pt,cls,n_perm)), header=True, index=True)
    
    endtime = timer()
    timePassed = (endtime-starttime)/60 # gives time in minutes
    
    if save == True:
        if act_filter == False:
            allperms.to_csv(os.path.join(path,'results','indiv_%iperms_41splits_%s.csv' % (n_perm,clsf)), header=True, index=True)
        elif act_filter == True:
            allperms.to_csv(os.path.join(path,'results','indiv_%iperms_41splits_%s_ACTFILTER.csv' % (n_perm,clsf)), header=True, index=True)
        
    print('Minutes passed: %.1f' % timePassed)    
    
    return allperms



In [48]:
permall = indiv_classification_perm('RF', features, False, 5, True)
permact = indiv_classification_perm('RF', features, True, 5, True)

/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


002 all predictions ready
012 all predictions ready
013 all predictions ready


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


014 all predictions ready
015 all predictions ready


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

063 all predictions ready


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd

065 all predictions ready
079 all predictions ready


/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jeroenhabets/Research/pd_sensors/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


090 all predictions ready
Minutes passed: 9.6


## Part 3: Group Model Classification (incl. activity-filter sub analysis)
Trains model for one patient on data from nineteen remaining patients, tests on data from the one specific patient.

In [72]:
def group_classification_true(clsf, features, feat_select, act_filter, save):
    
    starttime = timer()
    ## Data preparation for group classification
    pts = list(features.keys())
    keys = list(features[pts[0]].keys())
    if feat_select == 'all':
        sel = [k != 'medState' for k in keys]
        preds = list(compress(keys,sel))  
    elif feat_select == 'SVM':
        sel = [k[:3]=='SVM' for k in keys]
        preds = list(compress(keys,sel))
    elif feat_select == '4':
        sel = [k in ['SVM_maxAcc','SVM_coefVar','SVM_RMS','SVM_specPow_totalu4Hz'] for k in keys]
        preds = list(compress(keys,sel))
    
    metricNames = ['Accuracy', 'AUROC','F1_score', 'Precision', 'Recall', 'FPR'] # define metrics to check significancy for
    group_preds = pd.DataFrame(index=['pred'],columns=[pt+'_'+m for pt in pts for m in metricNames]) # 2d-df to save every pred per pt, per split
    predDict = {} # store selected features in dictionary for test/training selection based on patients
    for pt in pts:
        if act_filter:
            presize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==0)
            postsize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==1)
            minSize=np.min([presize,postsize]) # minimal number of rows per med-state, after actvity filter
            # selecting only rows with SVM_variance above threshold
            actData = features[pt][features[pt]['SVM_variance'] > 0.3].reset_index(drop=True)
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predDict[pt] = pd.concat([actData[actData['medState']==0][:minSize],actData[actData['medState']==1][:minSize]]).reset_index(drop=True)
        
        else:
            # minSize is shortest length of samples(rows) per pt-side of dataframe, on or off samples
            minSize = min(sum(features[pt]['medState']==0),sum(features[pt]['medState']==1))
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predDict[pt] = pd.concat([features[pt][features[pt]['medState']==0][:minSize],features[pt][features[pt]['medState']==1][:minSize]]).reset_index(drop=True)
    # data prepared: one dict with balanced features per patient (activity filtered or not)

    for pt in pts: ## Perform classification per patient that is tested
        testData = predDict[pt] # pt in loop is test-patient
        trainPts = pts.copy() # copy list to leave original pts list unchanged
        trainPts.remove(pt) # list with 19 training-patients
        trainData = pd.DataFrame(columns=predDict[pt].keys()) #create trainings df with other 19 patients
        for trainPt in trainPts:
            trainData = pd.concat([trainData,predDict[trainPt]]).reset_index(drop=True)
        scaler = StandardScaler() # standarize data, fit on trainingsdata (pre med)
        scaler.fit(trainData[trainData['medState']==0][preds]) # standardized only on pre-medication data
        # create x and y dataset 
        x_train = pd.DataFrame(data = scaler.transform(trainData[preds]), columns = preds)
        y_train = trainData['medState']
        x_test = pd.DataFrame(data = scaler.transform(testData[preds]), columns = preds)
        y_test = testData['medState']
        ## Performing prediction
        if clsf == 'SV':
            sv = SVC(C=10, kernel='rbf', gamma=0.001, probability=True) # define classifier as indiv-optimized class
            sv.fit(x_train, y_train) 
            y_preds = sv.predict(x_test) # predicting
            y_probas = pd.DataFrame(data = sv.predict_proba(x_test), columns = sv.classes_) 
        elif clsf == 'RF':
            rf = RandomForestClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2,n_estimators=100)
            rf.fit(x_train, y_train) # fitting on 19 training patients
            y_preds = rf.predict(x_test) # predicting
            y_probas = pd.DataFrame(data = rf.predict_proba(x_test), columns = rf.classes_)
        y_true = y_test
        tn, fp, fn, tp = confusion_matrix(y_true,y_preds).ravel()
        group_preds.loc['pred'][pt+'_'+'Accuracy'] = (accuracy_score(y_true,y_preds))
        group_preds.loc['pred'][pt+'_'+'AUROC'] = (roc_auc_score(y_true, y_probas[1]))
        group_preds.loc['pred'][pt+'_'+'F1_score'] = (f1_score(y_true,y_preds))
        group_preds.loc['pred'][pt+'_'+'Precision'] = (precision_score(y_true, y_preds)) # precision/PPV
        group_preds.loc['pred'][pt+'_'+'Recall'] = (recall_score(y_true, y_preds)) # sensitivity/recall/TPR
        group_preds.loc['pred'][pt+'_'+'FPR'] = (fp / (fp+tn)) # false-positive, false-alarm rate
#         print(pt,'is performed')
    endtime = timer()
    proc_time = endtime-starttime
    print('%s, %s fts, act %s, ready, %.1f minutes passed.' % (clsf,feat_select, 
                                                               str(act_filter),proc_time/60))
    
    if save:
        if act_filter == False:
            group_preds.to_csv(os.path.join(path,'results','group_pred_%s_%sfts.csv' % (clsf, feat_select)), header=True, index=True)
        elif act_filter == True:
            group_preds.to_csv(os.path.join(path,'results','group_pred_%s_%sfts_ACTFILTER.csv' % (clsf,feat_select)), header=True, index=True)
            
    return group_preds



In [73]:
for cls in ['RF']:
    for ft_sel in ['all','4']:
        for act in [True,False]:
            group_classification_true(cls, features, ft_sel, act, True)
# SV, all fts, act True, ready, 0.6 minutes passed.
# SV, all fts, act False, ready, 1.1 minutes passed.
# SV, 4 fts, act True, ready, 0.1 minutes passed.
# SV, 4 fts, act False, ready, 0.2 minutes passed.

RF, all fts, act True, ready, 0.2 minutes passed.
RF, all fts, act False, ready, 0.3 minutes passed.
RF, 4 fts, act True, ready, 0.1 minutes passed.
RF, 4 fts, act False, ready, 0.1 minutes passed.


In [85]:
def group_classification_perm(clsf, features, act_filter, n_perm, save):
    
    starttime = timer()
    ## Data preparation for group classification
    pts = list(features.keys())
    keys = list(features[pts[0]].keys())
    sel = [k != 'medState' for k in keys]
    preds = list(compress(keys,sel))  
    
    metricNames = ['Accuracy', 'AUROC','F1_score', 'Precision', 'Recall', 'FPR'] # define metrics to check significancy for
    group_perms = pd.DataFrame(index=np.arange(n_perm),columns=[pt+'_'+m for pt in pts for m in metricNames]) # 2d-df to save every pred per pt, per split
    predDict = {} # store selected features in dictionary for test/training selection based on patients
    for pt in pts:
        if act_filter:
            presize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==0)
            postsize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==1)
            minSize=np.min([presize,postsize]) # minimal number of rows per med-state, after actvity filter
            # selecting only rows with SVM_variance above threshold
            actData = features[pt][features[pt]['SVM_variance'] > 0.3].reset_index(drop=True)
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predDict[pt] = pd.concat([actData[actData['medState']==0][:minSize],actData[actData['medState']==1][:minSize]]).reset_index(drop=True)
        
        else:
            # minSize is shortest length of samples(rows) per pt-side of dataframe, on or off samples
            minSize = min(sum(features[pt]['medState']==0),sum(features[pt]['medState']==1))
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predDict[pt] = pd.concat([features[pt][features[pt]['medState']==0][:minSize],features[pt][features[pt]['medState']==1][:minSize]]).reset_index(drop=True)
    # data prepared: one dict with balanced features per patient (activity filtered or not)

    for pt in pts: ## Perform classification per patient that is tested
        testData = predDict[pt] # pt in loop is test-patient
        trainPts = pts.copy() # copy list to leave original pts list unchanged
        trainPts.remove(pt) # list with 19 training-patients
        trainData = pd.DataFrame(columns=predDict[pt].keys()) #create trainings df with other 19 patients
        for trainPt in trainPts:
            trainData = pd.concat([trainData,predDict[trainPt]]).reset_index(drop=True)
        scaler = StandardScaler() # standarize data, fit on trainingsdata (pre med)
        scaler.fit(trainData[trainData['medState']==0][preds]) # standardized only on pre-medication data
        x_train = pd.DataFrame(data = scaler.transform(trainData[preds]), columns = preds)
        y_train = trainData['medState']
        x_test = pd.DataFrame(data = scaler.transform(testData[preds]), columns = preds)
        y_test = testData['medState']
        ## data prepared for randomisation and permuted classification
        permlist={} # dict to store lists for perm-scores per metric (for every test-pt again)
        for m in metricNames:
            permlist[m] = []
        ## Start actual permutations, shuffling of y-labels
        for p in np.arange(n_perm): # loops over number of permutations
            y_random_train = list(y_train.values) # set true labels again as starting list, y_train itself stays unchanged
            y_random_test = list(y_test.values)
            random.seed(p) # set different random seed for every shuffle loop
            random.shuffle(y_random_train) # shuffles labels in y_random
            random.shuffle(y_random_test)
            ## Performing prediction
            if clsf == 'SV':
                sv = SVC(C=10, kernel='rbf', gamma=0.001, probability=True) # define classifier as indiv-optimized class
                sv.fit(x_train, y_random_train) 
                y_preds = sv.predict(x_test) # predicting
                y_probas = pd.DataFrame(data = sv.predict_proba(x_test), columns = sv.classes_) 
            elif clsf == 'RF':
                rf = RandomForestClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2,n_estimators=100)
                rf.fit(x_train, y_random_train) # fitting on 19 training patients
                y_preds = rf.predict(x_test) # predicting
                y_probas = pd.DataFrame(data = rf.predict_proba(x_test), columns = rf.classes_)
            y_true = y_random_test
            tn, fp, fn, tp = confusion_matrix(y_true,y_preds).ravel()
            permlist['Accuracy'].append(accuracy_score(y_true,y_preds))
            permlist['AUROC'].append(roc_auc_score(y_true, y_probas[1]))
            permlist['F1_score'].append(f1_score(y_true,y_preds))
            permlist['Precision'].append(precision_score(y_true, y_preds)) # precision/PPV
            permlist['Recall'].append(recall_score(y_true, y_preds)) # sensitivity/recall/TPR
            permlist['FPR'].append(fp / (fp+tn)) # false-positive, false-alarm rate
        
        for metric in metricNames: # at end of 41-splits in one permutations, save mean scores in outcome df
            group_perms[pt+'_'+metric] = permlist[metric]    
        print(pt,'all predictions ready')
    
    endtime = timer()
    timePassed = (endtime-starttime)/60 # gives time in minutes
    if save == True:
        if act_filter == False:
            group_perms.to_csv(os.path.join(path,'results','group_%s_%iperms.csv' % (clsf,n_perm)), header=True, index=True)
        elif act_filter == True:
            group_perms.to_csv(os.path.join(path,'results','group_%s_%iperms_ACTFILTER.csv' % (clsf,n_perm)), header=True, index=True)
        
    print('Minutes passed: %.1f' % timePassed)   

    return group_perms

## PM activity filter litearture
#actThresholdCheck(accData, features, 'SVM_variance', .3, save=True)
# eye-balling of episode without movement: variance +/- 0.005
# Van Hilten '91: > 0.1 g
# Keijsers '06: > 0.05 m/s
# Mahadevan '20: Coef of Var > 0.01

In [87]:
for cls in ['SV','RF']:
    for act in [False,True]:
        group_classification_perm(cls, features, act, 5, True)

002 all predictions ready
012 all predictions ready
013 all predictions ready
014 all predictions ready
015 all predictions ready
017 all predictions ready
018 all predictions ready
023 all predictions ready
024 all predictions ready
038 all predictions ready
039 all predictions ready
043 all predictions ready
047 all predictions ready
051 all predictions ready
054 all predictions ready
058 all predictions ready
063 all predictions ready
065 all predictions ready
079 all predictions ready
090 all predictions ready
Minutes passed: 5.5
002 all predictions ready
012 all predictions ready
013 all predictions ready
014 all predictions ready
015 all predictions ready
017 all predictions ready
018 all predictions ready
023 all predictions ready
024 all predictions ready
038 all predictions ready
039 all predictions ready
043 all predictions ready
047 all predictions ready
051 all predictions ready
054 all predictions ready
058 all predictions ready
063 all predictions ready
065 all prediction

## Part 4: Sub analyses on influence on amount of training data for group model 


Only Group Model analyzed, individual data is not large enough to decrease amount of training data used per paraticipant.

Test data: 

always data from one same patient

Training data:

1: start with data from 1 other patient (randomly selected); 5 random repititions

2: data from 2 other patients; 5 random rep's

3: data from 3 other patients; 5 random rep's

..

19: data from all 19 other patients; only one repitition possible

In [8]:
def amount_training_data(features, clsf, act_filter, test_pts, save):
    '''
    Function calculates classification-outcomes for individual model approach with SVM classifier.
    Input: 
    - features-dataframe containing all patients, all 60-sec features, and medication-states.
    - ptList: contains pt names to analyse.
    - save: has to be binary, determining whether outcomes are written to results-folder.
    
    Output: table with mean outcome of individual model per patient (mean is over the 41 possible data-splits),
    including p-values of permutation-test. 
        
    SVM classifier using hyperparameters: C 10, gamma 0.001, kernel rbf.
    
    Discussionpoints: standarisation, per patient, or per combi? Now: per combi.

    '''
    starttime = timer()
    pts = list(features.keys())
    preds = list(features[pts[0]].keys()) # define pred's
    preds.remove('medState')
    
    metrics = ['Accuracy', 'AUROC', ] 
    outcomeCols = [metr + '_mean' for metr in metrics]
    outcomeCols.extend([metr + '_sd' for metr in metrics])
    outcomeCols = [col + '_n%i' %npt for npt in np.arange(1,20) for col in outcomeCols]
    outcomes = pd.DataFrame(index=pts, columns=outcomeCols) # store final results
    ## Preparing data
    predDict={} # dict to store prediction-ready data per pt
    for pt in pts: 
        if act_filter:
            presize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==0)
            postsize = np.sum(features[pt][features[pt]['SVM_variance'] > 0.3]['medState']==1)
            minSize=np.min([presize,postsize]) # minimal number of rows per med-state, after actvity filter
            # selecting only rows with SVM_variance above threshold
            actData = features[pt][features[pt]['SVM_variance'] > 0.3].reset_index(drop=True)
            # create predData per pt, per side, with balanced rows pre and post med (minSize)
            predDict[pt] = pd.concat([actData[actData['medState']==0][:minSize],actData[actData['medState']==1][:minSize]]).reset_index(drop=True)
        else:
            minSize = min(sum(features[pt]['medState']==0),sum(features[pt]['medState']==1)) # minSize is shortest length of samples(rows) per pt-side of dataframe, on or off samples
            predDict[pt] = pd.concat([features[pt][features[pt]['medState']==0][:minSize],features[pt][features[pt]['medState']==1][:minSize]]).reset_index(drop=True)
    ## data splitting with increasing number of training patients
    for test_pt in test_pts: # loop over inserted patients to test
        for nPts in np.arange(1,20): # loop over number of training pt's [1-19]
            n_reps = 5 # number of repetitions, for every number of training pt's if possible
            if nPts == 19: # exceptions where 5 different repetitions are not possible
                n_reps = 1
            elif nPts == 18:
                n_reps = 2
            elif nPts == 17:
                n_reps = 3
            elif nPts == 16:
                n_reps = 4
            other_pts = list(features.keys()).copy() # list with all potential training pt's
            other_pts.remove(test_pt)
            acc_list = [] # list to store results (max5) per number of training pts
            auc_list = []
            for rep in np.arange(n_reps):
                random.seed(rep) # make sure every repetitions has always the same random.seed
                trainPts = random.sample(other_pts,nPts) # takes random sample of n=nPts, from other other_pts
                # make one df of data of trainings patients
                for n,train_pt in enumerate(trainPts):
                    if n == 0:
                        trainData = predDict[train_pt]
                    else:
                        trainData = pd.concat([trainData,predDict[train_pt]]).reset_index(drop=True)
                # standarize data 
                scaler = StandardScaler()
                scaler.fit(trainData[trainData['medState']==0][preds]) # standardized only on pre-medication data
                # create x and y dataset for this repetition, of this nr of trainingpts, for this test-patient
                x_train = pd.DataFrame(data = scaler.transform(trainData[preds]), columns = preds)
                y_train = trainData['medState']
                x_test = pd.DataFrame(data = scaler.transform(predDict[test_pt][preds]), columns = preds)
                y_test = predDict[test_pt]['medState']
                ## Performing prediction            
                if clsf == 'SV':
                    sv = SVC(C=10, kernel='rbf', gamma=0.001, probability=True) # define classifier as indiv-optimized class
                    sv.fit(x_train, y_train) 
                    y_preds = sv.predict(x_test) # predicting
                    y_probas = pd.DataFrame(data = sv.predict_proba(x_test), columns = sv.classes_) 
                elif clsf == 'RF':
                    rf = RandomForestClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2,n_estimators=100)
                    rf.fit(x_train, y_train) # fitting on 19 training patients
                    y_preds = rf.predict(x_test) # predicting
                    y_probas = pd.DataFrame(data = rf.predict_proba(x_test), columns = rf.classes_)
                y_true = y_test
    #             tn, fp, fn, tp = confusion_matrix(y_true,y_preds).ravel()
                acc_list.append(accuracy_score(y_true,y_preds))
                auc_list.append(roc_auc_score(y_true, y_probas[1]))
    #             indiv_outcomes.iloc[n_split]['F1_score'] = f1_score(y_true,y_preds)
    #             indiv_outcomes.iloc[n_split]['Precision'] = precision_score(y_true, y_preds) # precision/PPV
    #             indiv_outcomes.iloc[n_split]['Recall'] = recall_score(y_true, y_preds) # sensitivity/recall/TPR
    #             indiv_outcomes.iloc[n_split]['FPR'] = fp / (fp+tn) # false-alarm rate
            outcomes.loc[test_pt]['Accuracy_mean_n%i' % nPts] = np.mean(acc_list) # at end of 5 rep's            
            outcomes.loc[test_pt]['Accuracy_sd_n%i' % nPts] = np.std(acc_list)
            outcomes.loc[test_pt]['AUROC_mean_n%i' % nPts] = np.mean(auc_list)
            outcomes.loc[test_pt]['AUROC_sd_n%i' % nPts] = np.std(auc_list)
        print('Test patient',test_pt,'ready')    
    endtime = timer()
    print('Time passed: %.1f minutes' % ((endtime-starttime)/60)) # gives time in minutes
    
    if save == True:
        if act_filter:
            outcomes.to_csv(os.path.join(path,'results/preds_%s_nr_train_pts_actfilter.csv' % clsf), header=True, index=True)
        else:
            outcomes.to_csv(os.path.join(path,'results/preds_%s_nr_train_pts.csv' % clsf), header=True, index=True)
    
    return outcomes



In [10]:
allPts = list(features.keys())
test_pts = allPts
for cls in ['SV']:#'RF',
    amount_training_data(features, cls, True, test_pts, True)

Test patient 002 ready
Test patient 012 ready
Test patient 013 ready
Test patient 014 ready
Test patient 015 ready
Test patient 017 ready
Test patient 018 ready
Test patient 023 ready
Test patient 024 ready
Test patient 038 ready
Test patient 039 ready
Test patient 043 ready
Test patient 047 ready
Test patient 051 ready
Test patient 054 ready
Test patient 058 ready
Test patient 063 ready
Test patient 065 ready
Test patient 079 ready
Test patient 090 ready
Time passed: 16.4 minutes


In [8]:
## ADDING INDIVIDUAL PERMUTATIONS TO GROUP FILE

listdir(os.path.join(path,'results/perms/temp'))

['043_indiv_SV_allfts_actfilter_5000perms.csv',
 '039_indiv_SV_allfts_actfilter_5000perms.csv',
 '002_indiv_SV_allfts_actfilter_5000perms.csv',
 '012_group_SV_allfts_actfilter_5000perms.csv',
 '013_group_SV_allfts_actfilter_5000perms.csv',
 '038_indiv_SV_allfts_actfilter_5000perms.csv',
 '079_indiv_SV_allfts_actfilter_5000perms.csv',
 '014_group_SV_allfts_actfilter_5000perms.csv',
 '047_indiv_SV_allfts_actfilter_5000perms.csv',
 '051_indiv_SV_allfts_actfilter_5000perms.csv',
 '065_indiv_SV_allfts_actfilter_5000perms.csv',
 '024_indiv_SV_allfts_actfilter_5000perms.csv',
 '013_indiv_SV_allfts_actfilter_5000perms.csv',
 '090_indiv_SV_allfts_actfilter_5000perms.csv',
 '012_indiv_SV_allfts_actfilter_5000perms.csv',
 '002_group_SV_allfts_actfilter_5000perms.csv',
 '018_indiv_SV_allfts_actfilter_5000perms.csv',
 '023_indiv_SV_allfts_actfilter_5000perms.csv',
 '017_indiv_SV_allfts_actfilter_5000perms.csv',
 '058_indiv_SV_allfts_actfilter_5000perms.csv',
 '063_indiv_SV_allfts_actfilter_5000perm

In [41]:

f = pd.read_csv(os.path.join(path,'results/perms/indiv_SV_4fts_actfilter_5000perms.csv'), index_col=0)
set([k[:3] for k in f.keys()])
    

{'015',
 '017',
 '018',
 '023',
 '024',
 '038',
 '039',
 '043',
 '047',
 '051',
 '054',
 '058',
 '063',
 '065',
 '079',
 '090'}

In [44]:
# main = pd.read_csv(os.path.join(path,'results/perms/002_043_indiv_SV_allfts_5000perms.csv'), index_col=0)
# main

# for pt in ['047','051','054','058','063','065','079','090']:
#     temp = pd.read_csv(os.path.join(path,'results/perms/%s_indiv_SV_allfts_5000perms.csv'%pt), index_col=0)
#     for col in list(main.keys()):
#         if col[:3] == pt:
#             main[col] = temp[col]
# main.to_csv(os.path.join(path,'results/perms/indiv_SV_allfts_5000perms.csv'), index=True)

# model = 'indiv_SV_allfts_actfilter_5000perms'
# files = listdir(os.path.join(path,'results/perms/temp'))
# pts = np.sort(list(set([file[:3] for file in files])))
# first = pd.read_csv(os.path.join(path,'results/perms/temp/002_%s.csv' % model), index_col=0)
# main = pd.DataFrame(columns=[pt+var[3:] for pt in pts for var in first.keys() ])
# for file in files:
#     if file[4:-4] == model:
#         dat = pd.read_csv(os.path.join(path,'results/perms/temp/%s' % file), index_col=0)
#         for col in dat.keys():
#             main[col] = dat[col]
# main.to_csv(os.path.join(path,'results/perms/%s.csv' % model), index=True)
# main

# ## STILL TO DO: [002-014] for indiv SV 4fts actfilter (now filled with 0.5's)
# main = pd.read_csv(os.path.join(path,'results/perms/indiv_SV_4fts_actfilter_5000perms.csv'), index_col=0)
# model = 'indiv_SV_4fts_actfilter_5000perms'
# pts = set([k[:3] for k in main.keys()])
# pts_todo = ['002','012','013','014']
# for pt in pts_todo:
#     dat = pd.read_csv(os.path.join(path,'results/perms/temp/%s_%s.csv' % (pt,model)), index_col=0)
#     for c in dat.keys():
#         main[c] = dat[c]
# #     for col in set([c[3:] for c in main.keys()]):
# #         main[pt+col] = [0.5]*5000
           
# main.to_csv(os.path.join(path,'results/perms/%s.csv' % model), index=True)
